In [5]:
# Instalação de dados
!pip install mlflow scikit-learn pandas --upgrade --quiet
print("Bibliotecas instaladas.")

import mlflow
import pandas as pd
import io

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Dataset sorvete (CSV)
dados_csv_sorvete = """Temperature,Revenue
24.56,534.79
26.00,625.29
27.54,660.17
29.21,690.81
30.42,751.49
31.10,771.09
32.17,804.81
33.28,829.17
34.92,903.54
35.76,929.88
36.37,950.49
38.00,999.00
"""

# Cria e verifica o Dataframe
df_sorvetes = pd.read_csv(io.StringIO(dados_csv_sorvete))
print("\n---DADOS CARREGADOS ---")
print(df_sorvetes.head())


Bibliotecas instaladas.

---DADOS CARREGADOS ---
   Temperature  Revenue
0        24.56   534.79
1        26.00   625.29
2        27.54   660.17
3        29.21   690.81
4        30.42   751.49


In [2]:
# Configura o mlflow 
experiment_name = "regressao-vendas-sorvete-dp100"
mlflow.set_experiment(experiment_name)

print(f"Sucesso! O MLflow foi configurado.")
print(f"Seus resultados serão salvos no Expreimento: {experiment_name}")


Sucesso! O MLflow foi configurado.
Seus resultados serão salvos no Expreimento: regressao-vendas-sorvete-dp100


In [4]:
# TREINAMENTO E REGISTO DO MODELO
# 1. INICIAR O REGRISTRO
run = mlflow.start_run(run_name="Regressão_Linear_Simples_Explícita")
print("Regressão MLflow INICIADO.")

# ---------------------------------------------------------------------
# 2. SEPARAÇÃO DOS DADOS
# X (Feature/Entrada): O que usamos para prever (Temperatura). 
# Y (Target/Saída): O que queremos prever (Receita).
X = df_sorvetes[['Temperature']]
y = df_sorvetes['Revenue']

# Dividir dataset: 80% treino e 20% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Dados separados em treino e teste.")

# -----------------------------------------------------------------------
# 3. TREINAR O MODELO
model = LinearRegression()
model.fit(X_train, y_train)
print("Modelo de Regressão Linear treinado com sucesso.")

# ------------------------------------------------------------------------
# 4. AVALIAR O MODELO
y_pred = model.predict(X_test)

# Calcular as métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\Métricas finais: R2={r2:.4f}, MSE={mse:.2f}")

# -------------------------------------------------------------------------
# 5. REGISTRAR NO AZURE
print("\nRegistrando modelo, parâmetros e métricas no Azure...")

mlflow.log_param("algoritmo", "Regressão Linear")
mlflow.log_param("split_percent", 0.8)

mlflow.log_metric("R2_Score", r2)
mlflow.log_metric("MSE", mse)

mlflow.sklearn.log_model(
    sk_model=model, 
    artifact_path="modelo_sorvete_dp100", 
    registered_model_name="Vendas_Sorvete_DP100")

# ---------------------------------------------------------------------------
# 6. ENCERRAR O REGISTRO

mlflow.end_run()
print("\n--- REGISTRO CONCLUÍDO! ---")


Regressão MLflow INICIADO.
Dados separados em treino e teste.
Modelo de Regressão Linear treinado com sucesso.
\Métricas finais: R2=0.9911, MSE=324.22

Registrando modelo, parâmetros e métricas no Azure...


2025/11/14 13:17:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


MlflowException: API request to endpoint /api/2.0/mlflow/logged-models failed with error code 404 != 200. Response body: ''